In [1]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from matplotlib import pyplot as plt
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.preprocessing import timeseries_dataset_from_array

# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe


Using TensorFlow backend.


In [2]:
# df = pd.read_csv("model_data.csv")

In [1]:
def data():
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split

    df = pd.read_csv("model_data.csv")

    def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
        n_vars = 1 if type(data) is list else data.shape[1]
        df = pd.DataFrame(data)
        cols, names = list(), list()
        # input sequence (t-n, ... t-1)
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        # forecast sequence (t, t+1, ... t+n)
        for i in range(0, n_out):
            cols.append(df.shift(-i))
            if i == 0:
                names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
            else:
                names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        # put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)
        return agg

    values = df.drop(["Date"], axis=1).values
    values = values.astype("float32")
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)

    reframed = series_to_supervised(scaled, 1, 1)
    reframed.drop(reframed.columns[[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]], axis=1, inplace=True)


    reframe_values = reframed.values
    X = reframe_values[:,:-1]
    y = reframe_values[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    X_train = np.expand_dims(X_train, axis=1)
    X_test = np.expand_dims(X_test, axis=1)
    y_train = np.expand_dims(y_train, axis=1)
    y_test = np.expand_dims(y_test, axis=1)
    
    return X_train, y_train, X_test, y_test
    #Create a specific train split before 

In [15]:
# X_train.shape

(20354, 1, 15)

In [17]:
import numpy as numpy


(6785, 1, 15)

In [2]:
def create_model(X_train, X_test, y_train, y_test):

        import tensorflow as tf
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense
        from tensorflow.keras.layers import LSTM
        from hyperas.distributions import choice, uniform
        from hyperopt import STATUS_OK


        model = Sequential()
        model.add(LSTM({{choice([5, 10, 25, 50, 75, 100])}}, 
                input_shape=(X_train.shape[1], X_train.shape[2]))
                )

        model.add(Dense({{choice([10, 20, 50, 100])}}))
        model.add(Dense(1))
        model.compile(loss={{choice(["mae", "mse"])}}, 
                    optimizer={{choice(["adam", "sgd", "rmsprop"])}}, 
                    metrics=["mae", "mse"]
                    )
        e_stop =  tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, mode="min")

        result = model.fit(X_train, y_train, epochs=25, batch_size=60, validation_split=0.1, verbose=2, callbacks = [e_stop], shuffle=False)

        # print(result.history)

        validation_loss = np.amin(result.history['val_loss'])
        print('Best Validation loss of epoch:', validation_loss)
        return {'loss': validation_loss, 'status': STATUS_OK, 'model':model}

In [3]:
# if __name__ == '__main__':
from hyperas import optim
from hyperopt import Trials, tpe

best_run, best_model = optim.minimize(model=create_model,
                                        data=data,
                                        algo=tpe.suggest,
                                        max_evals=5,
                                        trials=Trials(),
                                        notebook_name = 'HyperParameter_Tuning')
                                        
X_train, y_train, X_test, y_test = data()
print('Evaluation of best performing model:')
print(best_model.evaluate(X_test, y_test))
print("Best Performing Model Hyper-Parameters:")
print(best_run)

Using TensorFlow backend.
>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import numpy as numpy
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from tensorflow.keras.layers import LSTM
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import STATUS_OK
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperopt import Trials, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperopt import Trials, tpe
except:
    pass

>>> Hyperas search space:



In [ ]:
if __name__ == '__main__':
    from hyperas import optim
    from hyperopt import Trials, tpe

    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name = 'BlackBox')
                                          
    X_train, y_train, X_test, y_test = data()
    print('Evaluation of best performing model:')
    print(best_model.evaluate(X_test, y_test))
    print("Best Performing Model Hyper-Parameters:")
    print(best_run)